In [1]:
import os , sys
sys.path.append(os.path.join(os.getcwd(),'..'))

In [2]:
import pandas as pd
from scripts.sentiment_labeler import assign_sentiment_label

from textblob import TextBlob


In [4]:
data = pd.read_csv('C:/kifiya ai/stock-market-predictions/src/data/raw_analyst_ratings.csv')

In [5]:
apple_data = pd.read_csv('../src/data/yfinance_data/AAPL_historical_data.csv')
amazon_data = pd.read_csv('../src/data/yfinance_data/AMZN_historical_data.csv')
google_data = pd.read_csv('../src/data/yfinance_data/GOOG_historical_data.csv')
meta_data = pd.read_csv('../src/data/yfinance_data/META_historical_data.csv')
microsoft_data = pd.read_csv('../src/data/yfinance_data/MSFT_historical_data.csv')
nvidia_data = pd.read_csv('../src/data/yfinance_data/NVDA_historical_data.csv')
tesla_data = pd.read_csv('../src/data/yfinance_data/TSLA_historical_data.csv')

In [17]:
data['date'] = pd.to_datetime(data['date'],  errors='coerce')
data['sentiment_label'] = data['headline'].apply(lambda headline: assign_sentiment_label(TextBlob(headline).sentiment.polarity))
data.head()


,Unnamed: 0,headline,url,publisher,date,stock,sentiment_label
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,Neutral
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A,Neutral
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A,Neutral
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A,Neutral
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A,Neutral


In [44]:
# Ensure both columns are in datetime format
data['date'] = pd.to_datetime(data['date'], errors='coerce').dt.date
tesla_data['Date'] = pd.to_datetime(tesla_data['Date'], errors='coerce').dt.date

# Filter the 'data' table where the stock column is 'TSLA'
tes_data = data[data['stock'] == 'AMZN']

# Merge the filtered table on the cleaned date columns
merged_tesla_table = pd.merge(tes_data, tesla_data, left_on='date', right_on='Date', how='inner')

# Drop the redundant 'Date' column after merging
merged_tesla_table = merged_tesla_table.drop(columns=['Date'])

# Calculate percentage change in closing prices
merged_tesla_table['Stock_Return'] = merged_tesla_table['Adj Close'].pct_change()

# Map Sentiment Scores to Numerical Values
# Assuming 'sentiment_label' column contains "Positive", "Neutral", or "Negative"
sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
merged_tesla_table['sentiment'] = merged_tesla_table['sentiment_label'].map(sentiment_mapping)

# Drop rows with NaN values in sentiment or Stock_Return
cleaned_data = merged_tesla_table.dropna(subset=['sentiment', 'Stock_Return'])

# Calculate correlation
correlation = cleaned_data[['sentiment', 'Stock_Return']].corr().iloc[0, 1]
print(f'Correlation between news sentiment and stock returns: {correlation}')


Correlation between news sentiment and stock returns: nan


In [43]:
# Ensure both columns are in datetime format
data['date'] = pd.to_datetime(data['date'], errors='coerce').dt.date
amazon_data['Date'] = pd.to_datetime(amazon_data['Date'], errors='coerce').dt.date

# Filter the 'data' table where the stock column is 'TSLA'
amaz_data = data[data['stock'] == 'AMZN']

# Merge the filtered table on the cleaned date columns
merged_amazon_table = pd.merge(amaz_data, amazon_data, left_on='date', right_on='Date', how='inner')

# Drop the redundant 'Date' column after merging
merged_amazon_table = merged_amazon_table.drop(columns=['Date'])

# Calculate percentage change in closing prices
merged_amazon_table['Stock_Return'] = merged_amazon_table['Adj Close'].pct_change()

# Map Sentiment Scores to Numerical Values
# Assuming 'sentiment_label' column contains "Positive", "Neutral", or "Negative"
sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
merged_amazon_table['sentiment'] = merged_amazon_table['sentiment_label'].map(sentiment_mapping)

# Drop rows with NaN values in sentiment or Stock_Return
cleaned_data = merged_amazon_table.dropna(subset=['sentiment', 'Stock_Return'])

# Calculate correlation
correlation = cleaned_data[['sentiment', 'Stock_Return']].corr().iloc[0, 1]
print(f'Correlation between news sentiment and stock returns: {correlation}')


Correlation between news sentiment and stock returns: 0.22941573387056174


In [47]:
# Ensure both columns are in datetime format
data['date'] = pd.to_datetime(data['date'], errors='coerce').dt.date
apple_data['Date'] = pd.to_datetime(apple_data['Date'], errors='coerce').dt.date

# Filter the 'data' table where the stock column is 'TSLA'
app_data = data[data['stock'] == 'AAPL']

# Merge the filtered table on the cleaned date columns
merged_apple_table = pd.merge(app_data, apple_data, left_on='date', right_on='Date', how='inner')

# Drop the redundant 'Date' column after merging
merged_apple_table = merged_apple_table.drop(columns=['Date'])

# Calculate percentage change in closing prices
merged_apple_table['Stock_Return'] = merged_apple_table['Adj Close'].pct_change()

# Map Sentiment Scores to Numerical Values
# Assuming 'sentiment_label' column contains "Positive", "Neutral", or "Negative"
sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
merged_apple_table['sentiment'] = merged_apple_table['sentiment_label'].map(sentiment_mapping)

# Drop rows with NaN values in sentiment or Stock_Return
cleaned_data = merged_apple_table.dropna(subset=['sentiment', 'Stock_Return'])

# Calculate correlation
correlation = cleaned_data[['sentiment', 'Stock_Return']].corr().iloc[0, 1]
print(f'Correlation between news sentiment and stock returns: {correlation}')


Correlation between news sentiment and stock returns: -0.31622776601683794


In [48]:
# Ensure both columns are in datetime format
data['date'] = pd.to_datetime(data['date'], errors='coerce').dt.date
microsoft_data['Date'] = pd.to_datetime(microsoft_data['Date'], errors='coerce').dt.date

# Filter the 'data' table where the stock column is 'TSLA'
mic_data = data[data['stock'] == 'MSFT']

# Merge the filtered table on the cleaned date columns
merged_microsoft_table = pd.merge(mic_data, microsoft_data, left_on='date', right_on='Date', how='inner')

# Drop the redundant 'Date' column after merging
merged_microsoft_table = merged_microsoft_table.drop(columns=['Date'])

# Calculate percentage change in closing prices
merged_microsoft_table['Stock_Return'] = merged_microsoft_table['Adj Close'].pct_change()

# Map Sentiment Scores to Numerical Values
# Assuming 'sentiment_label' column contains "Positive", "Neutral", or "Negative"
sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
merged_microsoft_table['sentiment'] = merged_microsoft_table['sentiment_label'].map(sentiment_mapping)

# Drop rows with NaN values in sentiment or Stock_Return
cleaned_data = merged_microsoft_table.dropna(subset=['sentiment', 'Stock_Return'])

# Calculate correlation
correlation = cleaned_data[['sentiment', 'Stock_Return']].corr().iloc[0, 1]
print(f'Correlation between news sentiment and stock returns: {correlation}')


Correlation between news sentiment and stock returns: nan


In [49]:
# Ensure both columns are in datetime format
data['date'] = pd.to_datetime(data['date'], errors='coerce').dt.date
google_data['Date'] = pd.to_datetime(google_data['Date'], errors='coerce').dt.date

# Filter the 'data' table where the stock column is 'TSLA'
goog_data = data[data['stock'] == 'GOOG']

# Merge the filtered table on the cleaned date columns
merged_google_table = pd.merge(goog_data, google_data, left_on='date', right_on='Date', how='inner')

# Drop the redundant 'Date' column after merging
merged_google_table = merged_google_table.drop(columns=['Date'])

# Calculate percentage change in closing prices
merged_google_table['Stock_Return'] = merged_google_table['Adj Close'].pct_change()

# Map Sentiment Scores to Numerical Values
# Assuming 'sentiment_label' column contains "Positive", "Neutral", or "Negative"
sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
merged_google_table['sentiment'] = merged_google_table['sentiment_label'].map(sentiment_mapping)

# Drop rows with NaN values in sentiment or Stock_Return
cleaned_data = merged_google_table.dropna(subset=['sentiment', 'Stock_Return'])

# Calculate correlation
correlation = cleaned_data[['sentiment', 'Stock_Return']].corr().iloc[0, 1]
print(f'Correlation between news sentiment and stock returns: {correlation}')


Correlation between news sentiment and stock returns: -0.08308662177227347


In [50]:
# Ensure both columns are in datetime format
data['date'] = pd.to_datetime(data['date'], errors='coerce').dt.date
nvidia_data['Date'] = pd.to_datetime(nvidia_data['Date'], errors='coerce').dt.date

# Filter the 'data' table where the stock column is 'TSLA'
nvi_data = data[data['stock'] == 'GOOG']

# Merge the filtered table on the cleaned date columns
merged_nvidia_table = pd.merge(nvi_data, nvidia_data, left_on='date', right_on='Date', how='inner')

# Drop the redundant 'Date' column after merging
merged_nvidia_table = merged_nvidia_table.drop(columns=['Date'])

# Calculate percentage change in closing prices
merged_nvidia_table['Stock_Return'] = merged_nvidia_table['Adj Close'].pct_change()

# Map Sentiment Scores to Numerical Values
# Assuming 'sentiment_label' column contains "Positive", "Neutral", or "Negative"
sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
merged_nvidia_table['sentiment'] = merged_nvidia_table['sentiment_label'].map(sentiment_mapping)

# Drop rows with NaN values in sentiment or Stock_Return
cleaned_data = merged_nvidia_table.dropna(subset=['sentiment', 'Stock_Return'])

# Calculate correlation
correlation = cleaned_data[['sentiment', 'Stock_Return']].corr().iloc[0, 1]
print(f'Correlation between news sentiment and stock returns: {correlation}')


Correlation between news sentiment and stock returns: -0.2635131877859457
